# Chapter 19: File I/O and Serialization

Almost every real‑world application needs to interact with the file system – reading configuration files, saving user data, logging, or processing uploaded documents. In addition, you often need to convert objects in memory to a format that can be stored or transmitted, and then reconstruct them later. This is **serialization**.

In this chapter, you'll learn:

- How to work with files and directories using the `System.IO` namespace.
- High‑level file access with `File` class methods.
- Low‑level stream programming with `FileStream`, `StreamReader`, `StreamWriter`, and binary readers/writers.
- Asynchronous file I/O for better performance.
- JSON serialization with `System.Text.Json` – the modern, high‑performance serializer.
- XML serialization with `XmlSerializer` for legacy interoperability.
- How to control serialization with attributes.
- A practical example: building a simple note‑taking application that persists data to a JSON file.
- Best practices and common pitfalls.

By the end, you'll be able to persistently store and retrieve data, and choose the right serialization format for your needs.

---

## 19.1 The `System.IO` Namespace

The `System.IO` namespace contains classes for working with files, directories, and streams. The most commonly used types are:

- `File` – static methods for creating, copying, deleting, and moving files.
- `Directory` – static methods for working with directories.
- `Path` – static methods for manipulating file and directory paths.
- `FileStream` – a stream for reading/writing bytes to/from a file.
- `StreamReader` / `StreamWriter` – for reading/writing text with encodings.
- `BinaryReader` / `BinaryWriter` – for reading/writing primitive data types in binary form.
- `FileSystemWatcher` – to monitor file system changes.

---

## 19.2 High‑Level File Access with the `File` Class

For simple file operations, the `File` class provides convenient static methods that handle opening, reading, writing, and closing files in one line.

### Reading All Text

```csharp
string content = File.ReadAllText(@"C:\temp\file.txt");
Console.WriteLine(content);
```

### Writing All Text

```csharp
string content = "Hello, world!";
File.WriteAllText(@"C:\temp\file.txt", content);
```

If the file exists, it is overwritten. Use `File.AppendAllText` to append.

### Reading All Lines as an Array

```csharp
string[] lines = File.ReadAllLines(@"C:\temp\file.txt");
foreach (string line in lines)
{
    Console.WriteLine(line);
}
```

### Writing an Array of Lines

```csharp
string[] lines = { "First line", "Second line" };
File.WriteAllLines(@"C:\temp\file.txt", lines);
```

### Checking Existence, Copying, Moving

```csharp
if (File.Exists(@"C:\temp\file.txt"))
{
    File.Copy(@"C:\temp\file.txt", @"C:\temp\backup.txt");
    File.Move(@"C:\temp\file.txt", @"C:\temp\renamed.txt");
    File.Delete(@"C:\temp\backup.txt");
}
```

These high‑level methods are great for simple scenarios, but they load the entire file into memory. For large files, you should use streams.

---

## 19.3 Working with Streams

A **stream** is a sequence of bytes. You can read from or write to a stream sequentially. Streams are the foundation of all I/O in .NET.

### `FileStream`

`FileStream` provides raw byte access to files.

```csharp
using (FileStream fs = new FileStream(@"C:\temp\data.bin", FileMode.Create))
{
    byte[] data = { 0x48, 0x65, 0x6C, 0x6C, 0x6F }; // "Hello" in ASCII
    fs.Write(data, 0, data.Length);
}

using (FileStream fs = new FileStream(@"C:\temp\data.bin", FileMode.Open))
{
    byte[] buffer = new byte[fs.Length];
    fs.Read(buffer, 0, buffer.Length);
    Console.WriteLine(System.Text.Encoding.ASCII.GetString(buffer)); // Hello
}
```

### `StreamReader` and `StreamWriter` for Text

These classes work on top of a stream and handle text encoding (UTF‑8 by default).

```csharp
using (StreamWriter writer = new StreamWriter(@"C:\temp\file.txt"))
{
    writer.WriteLine("First line");
    writer.WriteLine("Second line");
}

using (StreamReader reader = new StreamReader(@"C:\temp\file.txt"))
{
    string line;
    while ((line = reader.ReadLine()) != null)
    {
        Console.WriteLine(line);
    }
}
```

Always use `using` (or `await using`) to ensure the stream is closed, even if an exception occurs.

### `BinaryReader` and `BinaryWriter`

These classes read and write primitive data types in binary format, which is compact and fast.

```csharp
using (BinaryWriter writer = new BinaryWriter(File.Open(@"C:\temp\data.bin", FileMode.Create)))
{
    writer.Write(42);                 // int
    writer.Write(3.14159);            // double
    writer.Write("Hello");             // string (length‑prefixed)
}

using (BinaryReader reader = new BinaryReader(File.Open(@"C:\temp\data.bin", FileMode.Open)))
{
    int i = reader.ReadInt32();
    double d = reader.ReadDouble();
    string s = reader.ReadString();
    Console.WriteLine($"{i}, {d}, {s}"); // 42, 3.14159, Hello
}
```

Binary formats are not human‑readable but are efficient and preserve exact data.

---

## 19.4 Asynchronous File I/O

All the stream classes provide asynchronous methods (`ReadAsync`, `WriteAsync`, etc.). Use them in UI or server applications to avoid blocking threads.

```csharp
public async Task ProcessFileAsync(string path)
{
    using (StreamReader reader = new StreamReader(path))
    {
        string content = await reader.ReadToEndAsync();
        Console.WriteLine(content);
    }
}
```

For writing:

```csharp
public async Task SaveFileAsync(string path, string content)
{
    using (StreamWriter writer = new StreamWriter(path))
    {
        await writer.WriteAsync(content);
    }
}
```

The high‑level `File` class also has async methods: `ReadAllTextAsync`, `WriteAllTextAsync`, etc.

---

## 19.5 JSON Serialization with `System.Text.Json`

JSON is the most popular data interchange format today. In .NET Core 3.0 and later, the recommended serializer is `System.Text.Json`. It's fast, allocates little memory, and is fully integrated into ASP.NET Core.

### Basic Serialization and Deserialization

```csharp
using System.Text.Json;

public class Person
{
    public string Name { get; set; }
    public int Age { get; set; }
    public string City { get; set; }
}

// Serialize an object to a JSON string
Person person = new Person { Name = "Alice", Age = 30, City = "New York" };
string json = JsonSerializer.Serialize(person);
Console.WriteLine(json); // {"Name":"Alice","Age":30,"City":"New York"}

// Deserialize from JSON string
Person deserialized = JsonSerializer.Deserialize<Person>(json);
Console.WriteLine(deserialized.Name); // Alice
```

### Serializing to a File

```csharp
Person person = new Person { Name = "Alice", Age = 30, City = "New York" };

// Write to file
string json = JsonSerializer.Serialize(person);
File.WriteAllText("person.json", json);

// Read from file
string jsonFromFile = File.ReadAllText("person.json");
Person loaded = JsonSerializer.Deserialize<Person>(jsonFromFile);
```

### Controlling Serialization with Options

You can customize serialization using `JsonSerializerOptions`.

```csharp
var options = new JsonSerializerOptions
{
    WriteIndented = true, // Pretty print
    PropertyNamingPolicy = JsonNamingPolicy.CamelCase // camelCase property names
};

string json = JsonSerializer.Serialize(person, options);
/*
{
  "name": "Alice",
  "age": 30,
  "city": "New York"
}
*/
```

### Handling Polymorphism and Complex Types

By default, `System.Text.Json` does not include type information. For polymorphic serialization, you may need to use custom converters or consider other libraries like Newtonsoft.Json.

---

## 19.6 XML Serialization with `XmlSerializer`

For legacy systems or specific requirements (e.g., SOAP web services), you might need XML serialization. .NET provides `XmlSerializer` in the `System.Xml.Serialization` namespace.

```csharp
using System.Xml.Serialization;

[XmlRoot("Person")]
public class Person
{
    [XmlElement("Name")]
    public string Name { get; set; }

    [XmlElement("Age")]
    public int Age { get; set; }

    [XmlElement("City")]
    public string City { get; set; }
}

// Serialize
Person person = new Person { Name = "Alice", Age = 30, City = "New York" };
XmlSerializer serializer = new XmlSerializer(typeof(Person));
using (StringWriter writer = new StringWriter())
{
    serializer.Serialize(writer, person);
    string xml = writer.ToString();
    Console.WriteLine(xml);
}

// Deserialize
using (StringReader reader = new StringReader(xml))
{
    Person deserialized = (Person)serializer.Deserialize(reader);
}
```

XML serialization requires a public parameterless constructor. You can control the output with attributes like `[XmlElement]`, `[XmlAttribute]`, `[XmlIgnore]`, etc.

---

## 19.7 Controlling Serialization with Attributes

Both JSON and XML serializers use attributes to customize behavior.

### JSON Attributes (System.Text.Json)

```csharp
using System.Text.Json.Serialization;

public class Person
{
    [JsonPropertyName("fullName")]
    public string Name { get; set; }

    [JsonIgnore]
    public int InternalId { get; set; }

    public string City { get; set; }
}
```

### XML Attributes

```csharp
public class Person
{
    [XmlAttribute("id")]
    public int Id { get; set; }

    [XmlElement("FullName")]
    public string Name { get; set; }

    [XmlIgnore]
    public int Age { get; set; }
}
```

---

## 19.8 Putting It All Together: A Note‑Taking App

Let's build a simple console application that lets the user create notes and saves them to a JSON file. It demonstrates file I/O, JSON serialization, and a simple menu.

```csharp
using System;
using System.Collections.Generic;
using System.IO;
using System.Text.Json;
using System.Threading.Tasks;

namespace NoteApp
{
    public class Note
    {
        public string Title { get; set; }
        public string Content { get; set; }
        public DateTime CreatedAt { get; set; }
        public DateTime? ModifiedAt { get; set; }
    }

    class Program
    {
        private static readonly string DataFile = "notes.json";
        private static List<Note> _notes = new List<Note>();

        static async Task Main(string[] args)
        {
            await LoadNotesAsync();

            bool exit = false;
            while (!exit)
            {
                Console.Clear();
                Console.WriteLine("Note Taking App");
                Console.WriteLine("1. List notes");
                Console.WriteLine("2. Add note");
                Console.WriteLine("3. View note");
                Console.WriteLine("4. Delete note");
                Console.WriteLine("5. Exit");
                Console.Write("Choose: ");

                switch (Console.ReadLine())
                {
                    case "1":
                        ListNotes();
                        break;
                    case "2":
                        await AddNoteAsync();
                        break;
                    case "3":
                        ViewNote();
                        break;
                    case "4":
                        await DeleteNoteAsync();
                        break;
                    case "5":
                        exit = true;
                        break;
                }
            }
        }

        static async Task LoadNotesAsync()
        {
            if (File.Exists(DataFile))
            {
                try
                {
                    string json = await File.ReadAllTextAsync(DataFile);
                    _notes = JsonSerializer.Deserialize<List<Note>>(json) ?? new List<Note>();
                }
                catch (Exception ex)
                {
                    Console.WriteLine($"Error loading notes: {ex.Message}");
                }
            }
        }

        static async Task SaveNotesAsync()
        {
            try
            {
                var options = new JsonSerializerOptions { WriteIndented = true };
                string json = JsonSerializer.Serialize(_notes, options);
                await File.WriteAllTextAsync(DataFile, json);
            }
            catch (Exception ex)
            {
                Console.WriteLine($"Error saving notes: {ex.Message}");
            }
        }

        static void ListNotes()
        {
            Console.Clear();
            if (_notes.Count == 0)
            {
                Console.WriteLine("No notes.");
            }
            else
            {
                for (int i = 0; i < _notes.Count; i++)
                {
                    Console.WriteLine($"{i + 1}. {_notes[i].Title} (Created: {_notes[i].CreatedAt:yyyy-MM-dd})");
                }
            }
            Console.WriteLine("\nPress any key to continue...");
            Console.ReadKey();
        }

        static async Task AddNoteAsync()
        {
            Console.Clear();
            Console.Write("Title: ");
            string title = Console.ReadLine();
            Console.Write("Content: ");
            string content = Console.ReadLine();

            var note = new Note
            {
                Title = title,
                Content = content,
                CreatedAt = DateTime.Now
            };
            _notes.Add(note);
            await SaveNotesAsync();
        }

        static void ViewNote()
        {
            Console.Clear();
            Console.Write("Enter note number: ");
            if (int.TryParse(Console.ReadLine(), out int index) && index > 0 && index <= _notes.Count)
            {
                var note = _notes[index - 1];
                Console.WriteLine($"Title: {note.Title}");
                Console.WriteLine($"Created: {note.CreatedAt}");
                if (note.ModifiedAt.HasValue)
                    Console.WriteLine($"Modified: {note.ModifiedAt}");
                Console.WriteLine("---");
                Console.WriteLine(note.Content);
            }
            else
            {
                Console.WriteLine("Invalid number.");
            }
            Console.WriteLine("\nPress any key to continue...");
            Console.ReadKey();
        }

        static async Task DeleteNoteAsync()
        {
            Console.Clear();
            Console.Write("Enter note number to delete: ");
            if (int.TryParse(Console.ReadLine(), out int index) && index > 0 && index <= _notes.Count)
            {
                _notes.RemoveAt(index - 1);
                await SaveNotesAsync();
                Console.WriteLine("Note deleted.");
            }
            else
            {
                Console.WriteLine("Invalid number.");
            }
            Console.WriteLine("\nPress any key to continue...");
            Console.ReadKey();
        }
    }
}
```

**Explanation:**

- `Note` class holds a note with title, content, and timestamps.
- `_notes` is an in‑memory list.
- On startup, `LoadNotesAsync` reads the JSON file and deserializes it into the list.
- Any modification (add, delete) calls `SaveNotesAsync` to persist changes.
- `JsonSerializerOptions` with `WriteIndented` makes the JSON file human‑readable.
- The app uses async file I/O for better scalability (even in a console app, it's a good habit).

This example demonstrates a complete cycle: loading, modifying, and saving data with JSON serialization.

---

## 19.9 Best Practices and Common Pitfalls

### 1. Always Use `using` or `await using` for Streams

Ensure that files are closed properly, even if exceptions occur. The `using` statement guarantees disposal.

### 2. Prefer Async I/O in UI and Server Applications

Async prevents thread blocking, improving scalability and responsiveness.

### 3. Handle Exceptions Appropriately

File I/O is prone to errors (file not found, access denied, disk full). Always wrap file operations in try‑catch and provide meaningful feedback.

### 4. Use `Path.Combine` to Build Paths

Instead of string concatenation, use `Path.Combine` to handle directory separators correctly across platforms.

```csharp
string fullPath = Path.Combine(folder, fileName);
```

### 5. Choose the Right Serialization Format

- **JSON** – modern, human‑readable, widely supported. Default choice for most applications.
- **XML** – verbose but self‑describing; use when interoperating with legacy systems.
- **Binary** – compact and fast, but not human‑readable and may be version‑sensitive.

### 6. Be Mindful of File Encoding

When reading/writing text, specify an encoding (UTF‑8 is the default and recommended). For interoperability, you may need to detect or specify the correct encoding.

### 7. Avoid Hard‑Coding Paths

Use configuration files, environment variables, or `AppContext.BaseDirectory` to locate files relative to your application.

### 8. Test with Different Scenarios

- File already exists or doesn't exist.
- Read‑only files or insufficient permissions.
- Very large files.
- Concurrent access (multiple processes/threads).

### 9. Consider Using `FileStream` for Large Files

High‑level methods like `ReadAllText` load the entire file into memory, which may cause `OutOfMemoryException` for huge files. Use streams to process data incrementally.

### 10. Protect Sensitive Data

If you're serializing personal data, consider encryption. Never store passwords or secrets in plain text.

---

## 19.10 Chapter Summary

In this chapter, you've learned how to work with the file system and serialize data:

- The **`File`** and **`Directory`** classes provide simple static methods for common operations.
- **Streams** (`FileStream`, `StreamReader`, `StreamWriter`, `BinaryReader`, `BinaryWriter`) give you low‑level control and are essential for large files.
- **Asynchronous file I/O** keeps your applications responsive.
- **JSON serialization** with `System.Text.Json` is the modern way to persist and exchange structured data.
- **XML serialization** is available for legacy interoperability.
- **Attributes** let you control how serialization works.
- A practical note‑taking app tied everything together.

With these tools, you can build applications that store and retrieve data persistently, whether it's user settings, documents, or complex object graphs.

In the next chapter, **Attributes and Reflection**, we'll explore how to add metadata to your code with attributes and inspect that metadata at runtime using reflection. This powerful combination enables features like dependency injection, serialization, and custom frameworks.

**Exercises:**

1. Write a program that reads a CSV file and displays its contents in a table format. Handle the case where the file doesn't exist.
2. Create a `ConfigurationManager` class that reads and writes application settings from a JSON file. Include methods to get/set values of different types (string, int, bool).
3. Serialize a list of `Product` objects (with `Name`, `Price`, `Category`) to both JSON and XML. Compare the file sizes.
4. Implement a simple logger that writes log messages to a file asynchronously. Use a `StreamWriter` with auto‑flush.
5. Build a file splitter/merger: split a large file into chunks of a given size, and then merge them back. Use binary streams.

Now, get ready to dive into metadata and reflection in Chapter 20!